# Model Building with BoFire

This notebooks shows how to setup and analyze models trained with BoFire. It is stil WIP.

## Imports

In [1]:
from bofire.data_models.domain.api import Inputs, Outputs
from bofire.data_models.surrogates.api import SingleTaskGPSurrogate, RandomForestSurrogate, MixedSingleTaskGPSurrogate, AnySurrogate, RandomForestSurrogate, EmpiricalSurrogate, MLPEnsemble
from bofire.benchmarks.single import Himmelblau
from bofire.benchmarks.multi import CrossCoupling
import bofire.surrogates.api as surrogates
import json
from bofire.data_models.enum import CategoricalEncodingEnum

from pydantic import parse_obj_as

/opt/homebrew/Caskroom/miniforge/base/envs/bofire/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Problem Setup

For didactic purposes, we sample data from a Himmelblau benchmark function and use them to train a SingleTaskGP.

In [2]:
benchmark = Himmelblau()
samples = benchmark.domain.inputs.sample(n=50)
experiments = benchmark.f(samples, return_complete=True)

experiments.head(10)

,x_1,x_2,y,valid_y
0,-4.189148,-0.843587,142.329504,1
1,-2.548556,-2.351760,63.155820,1
2,-1.089940,-2.448055,154.706964,1
3,0.870975,-3.246683,201.393531,1
4,5.098456,5.981887,1587.949784,1
5,3.968824,3.817305,206.611937,1
6,3.432720,-3.051569,38.146483,1
7,-3.577077,0.099487,115.256187,1
8,0.776818,5.169825,447.728824,1
9,4.928474,-4.601601,440.418108,1


## Model Fitting


In [3]:
input_features = benchmark.domain.inputs
output_features = benchmark.domain.outputs

In [4]:
input_features.json()

'{"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "unit": null, "bounds": [-6.0, 6.0]}, {"type": "ContinuousInput", "key": "x_2", "unit": null, "bounds": [-6.0, 6.0]}]}'

In [5]:
output_features.json()

'{"type": "Outputs", "features": [{"type": "ContinuousOutput", "key": "y", "objective": {"type": "MinimizeObjective", "w": 1.0, "lower_bound": 0, "upper_bound": 1}, "unit": null}]}'

### Single Task GP

Generate the json spec

In [6]:
# we setup the data model, here a Single Task GP
surrogate_data = SingleTaskGPSurrogate(
    inputs=input_features,
    outputs=output_features
)

# we generate the json spec
jspec = surrogate_data.json()

jspec

'{"type": "SingleTaskGPSurrogate", "inputs": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "unit": null, "bounds": [-6.0, 6.0]}, {"type": "ContinuousInput", "key": "x_2", "unit": null, "bounds": [-6.0, 6.0]}]}, "outputs": {"type": "Outputs", "features": [{"type": "ContinuousOutput", "key": "y", "objective": {"type": "MinimizeObjective", "w": 1.0, "lower_bound": 0, "upper_bound": 1}, "unit": null}]}, "input_preprocessing_specs": {}, "dump": null, "kernel": {"type": "ScaleKernel", "base_kernel": {"type": "MaternKernel", "ard": true, "nu": 2.5, "lengthscale_prior": {"type": "GammaPrior", "concentration": 3.0, "rate": 6.0}}, "outputscale_prior": {"type": "GammaPrior", "concentration": 2.0, "rate": 0.15}}, "scaler": "NORMALIZE"}'

Load it from the spec

In [7]:
surrogate_data = parse_obj_as(AnySurrogate, json.loads(jspec))

Map it 

In [8]:
surrogate = surrogates.map(surrogate_data)

Fit it. This is not 100% finished. In the future we will call here hyperfit which will return the CV results etc. This has to be finished. So ignore this for now and just call fit.

In [9]:
surrogate.fit(experiments=experiments)

Dump it.

In [10]:
# dump it
dump = surrogate.dumps()

Make predictions.

In [11]:
# predict with it
df_predictions = surrogate.predict(experiments)
# transform to spec
predictions = surrogate.to_predictions(predictions=df_predictions)

Load again from spec and dump and make predictions.

In [12]:
surrogate_data = parse_obj_as(AnySurrogate, json.loads(jspec))
surrogate = surrogates.map(surrogate_data)
surrogate.loads(dump)

# predict with it
df_predictions2 = surrogate.predict(experiments)
# transform to spec
predictions2 = surrogate.to_predictions(predictions=df_predictions2)

# check for equality
predictions==predictions2


True

### Random Forest

Generate the json spec

In [14]:
# we setup the data model, here a Single Task GP
surrogate_data = RandomForestSurrogate(
    inputs=input_features,
    outputs=output_features,
    random_state=42
)

# we generate the json spec
jspec = surrogate_data.json()

jspec

'{"type": "RandomForestSurrogate", "inputs": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "unit": null, "bounds": [-6.0, 6.0]}, {"type": "ContinuousInput", "key": "x_2", "unit": null, "bounds": [-6.0, 6.0]}]}, "outputs": {"type": "Outputs", "features": [{"type": "ContinuousOutput", "key": "y", "objective": {"type": "MinimizeObjective", "w": 1.0, "lower_bound": 0, "upper_bound": 1}, "unit": null}]}, "input_preprocessing_specs": {}, "dump": null, "n_estimators": 100, "criterion": "squared_error", "max_depth": null, "min_samples_split": 2, "min_samples_leaf": 1, "min_weight_fraction_leaf": 0.0, "max_features": 1.0, "max_leaf_nodes": null, "min_impurity_decrease": 0.0, "bootstrap": true, "oob_score": false, "random_state": 42, "ccp_alpha": 0.0, "max_samples": null}'

In [15]:
# Load it from the spec
surrogate_data = parse_obj_as(AnySurrogate, json.loads(jspec))
# Map it 
surrogate = surrogates.map(surrogate_data)
# Fit it
surrogate.fit(experiments=experiments)
# dump it
dump = surrogate.dumps()
# predict with it
df_predictions = surrogate.predict(experiments)
# transform to spec
predictions = surrogate.to_predictions(predictions=df_predictions)

In [16]:
surrogate_data = parse_obj_as(AnySurrogate, json.loads(jspec))
surrogate = surrogates.map(surrogate_data)
surrogate.loads(dump)

# predict with it
df_predictions2 = surrogate.predict(experiments)
# transform to spec
predictions2 = surrogate.to_predictions(predictions=df_predictions2)

# check for equality
predictions==predictions2

True

### MLP Ensemble

Generate the json spec

In [17]:
# we setup the data model, here a Single Task GP
surrogate_data = MLPEnsemble(
    inputs=input_features,
    outputs=output_features,
    n_estimators=2
)

# we generate the json spec
jspec = surrogate_data.json()

jspec

'{"type": "MLPEnsemble", "inputs": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "unit": null, "bounds": [-6.0, 6.0]}, {"type": "ContinuousInput", "key": "x_2", "unit": null, "bounds": [-6.0, 6.0]}]}, "outputs": {"type": "Outputs", "features": [{"type": "ContinuousOutput", "key": "y", "objective": {"type": "MinimizeObjective", "w": 1.0, "lower_bound": 0, "upper_bound": 1}, "unit": null}]}, "input_preprocessing_specs": {}, "dump": null, "n_estimators": 2, "hidden_layer_sizes": [100], "activation": "relu", "dropout": 0.0, "batch_size": 10, "n_epochs": 200, "lr": 0.0001, "weight_decay": 0.0, "subsample_fraction": 1.0, "shuffle": true, "scaler": "NORMALIZE"}'

In [18]:
# Load it from the spec
surrogate_data = parse_obj_as(AnySurrogate, json.loads(jspec))
# Map it 
surrogate = surrogates.map(surrogate_data)
# Fit it
surrogate.fit(experiments=experiments)
# dump it
dump = surrogate.dumps()
# predict with it
df_predictions = surrogate.predict(experiments)
# transform to spec
predictions = surrogate.to_predictions(predictions=df_predictions)

/Users/j30607/sandbox/botorch/botorch/models/model.py:212: RuntimeWarning: Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.
  warnings.warn(


In [19]:
surrogate_data = parse_obj_as(AnySurrogate, json.loads(jspec))
surrogate = surrogates.map(surrogate_data)
surrogate.loads(dump)

# predict with it
df_predictions2 = surrogate.predict(experiments)
# transform to spec
predictions2 = surrogate.to_predictions(predictions=df_predictions2)

# check for equality
predictions==predictions2

/Users/j30607/sandbox/botorch/botorch/models/model.py:212: RuntimeWarning: Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.
  warnings.warn(


True

## Empirical Surrogate

The empirical model is special as it has per default no fit and you need cloudpickle. There can be empirical models which implement a fit, but for this they also have to inherit from `Trainable`. The current example is the default without any fit functionality.

In [20]:
from botorch.models.deterministic import DeterministicModel
from torch import Tensor

class HimmelblauModel(DeterministicModel):
    def __init__(self):
        super().__init__()
        self._num_outputs = 1

    def forward(self, X: Tensor) -> Tensor:
        return (
            (X[..., 0] ** 2 + X[..., 1] - 11.0) ** 2
            + (X[..., 0] + X[..., 1] ** 2 - 7.0) ** 2
        ).unsqueeze(-1)

In [21]:
# we setup the data model, here a Single Task GP
surrogate_data = EmpiricalSurrogate(
    inputs=input_features,
    outputs=output_features,
)

# we generate the json spec
jspec = surrogate_data.json()

jspec

'{"type": "EmpiricalSurrogate", "inputs": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "unit": null, "bounds": [-6.0, 6.0]}, {"type": "ContinuousInput", "key": "x_2", "unit": null, "bounds": [-6.0, 6.0]}]}, "outputs": {"type": "Outputs", "features": [{"type": "ContinuousOutput", "key": "y", "objective": {"type": "MinimizeObjective", "w": 1.0, "lower_bound": 0, "upper_bound": 1}, "unit": null}]}, "input_preprocessing_specs": {}, "dump": null}'

In [22]:
# Load it from the spec
surrogate_data = parse_obj_as(AnySurrogate, json.loads(jspec))
# Map it 
surrogate = surrogates.map(surrogate_data)
# attach the actual model to it
surrogate.model = HimmelblauModel()
# dump it
dump = surrogate.dumps()
# predict with it
df_predictions = surrogate.predict(experiments)
# transform to spec
predictions = surrogate.to_predictions(predictions=df_predictions)

In [23]:
surrogate_data = parse_obj_as(AnySurrogate, json.loads(jspec))
surrogate = surrogates.map(surrogate_data)
surrogate.loads(dump)

# predict with it
df_predictions2 = surrogate.predict(experiments)
# transform to spec
predictions2 = surrogate.to_predictions(predictions=df_predictions2)

# check for equality
predictions==predictions2

True

### Mixed GP

Generate the json spec

In [24]:
# we setup the data model, here a Single Task GP
surrogate_data = MixedSingleTaskGPSurrogate(
    inputs=input_features,
    outputs=output_features,
)

# we generate the json spec
jspec = surrogate_data.json()

jspec

ValidationError: 1 validation error for MixedSingleTaskGPSurrogate
input_preprocessing_specs
  MixedSingleTaskGPSurrogate can only be used if at least one one-hot encoded categorical feature is present. (type=value_error)

As expected this fails, as we do not have any categorical feature in the dataset. So we have to setup another problem.

In [25]:
benchmark = CrossCoupling()
samples = benchmark.domain.inputs.sample(n=50)
experiments = benchmark.f(samples, return_complete=True)

experiments.head(10)

,base_eq,t_res,temperature,base,catalyst,yield,cost,valid_cost,valid_yield
0,1.000929,640.729785,63.578259,DBU,AlPhos,0.860773,0.420104,1,1
1,1.848292,219.006008,91.659600,DBU,AlPhos,0.985511,0.421029,1,1
2,1.949376,239.685059,95.291403,DBU,tBuBrettPhos,1.001262,0.280399,1,1
3,1.215942,140.311163,95.770730,BTMG,tBuXPhos,0.942617,0.301352,1,1
4,1.401418,1076.142758,78.363599,TEA,tBuBrettPhos,0.016274,0.278780,1,1
5,2.205043,1047.478169,64.571328,DBU,tBuXPhos,0.788471,0.250647,1,1
6,2.011124,1434.620903,37.719431,BTMG,tBuXPhos,0.888348,0.336085,1,1
7,1.145737,434.494016,45.861940,DBU,tBuXPhos,0.218362,0.249490,1,1
8,1.998180,1758.219855,50.362649,TEA,tBuXPhos,0.119502,0.248967,1,1
9,1.370598,856.919234,54.740152,BTMG,tBuBrettPhos,1.000151,0.338138,1,1


In [28]:
# we setup the data model, here a Single Task GP
surrogate_data = MixedSingleTaskGPSurrogate(
    inputs=benchmark.domain.inputs,
    outputs=Outputs(features=[benchmark.domain.outputs.features[0]]),
    input_preprocessing_specs={"catalyst": CategoricalEncodingEnum.ONE_HOT}
)

# we generate the json spec
jspec = surrogate_data.json()

jspec

'{"type": "MixedSingleTaskGPSurrogate", "inputs": {"type": "Inputs", "features": [{"type": "CategoricalDescriptorInput", "key": "catalyst", "categories": ["tBuXPhos", "tBuBrettPhos", "AlPhos"], "allowed": [true, true, true], "descriptors": ["area_cat", "M2_cat"], "values": [[460.7543, 67.2057], [518.8408, 89.8738], [819.933, 129.0808]]}, {"type": "CategoricalDescriptorInput", "key": "base", "categories": ["TEA", "TMG", "BTMG", "DBU"], "allowed": [true, true, true, true], "descriptors": ["area", "M2"], "values": [[162.2992, 25.8165], [165.5447, 81.4847], [227.3523, 30.554], [192.4693, 59.8367]]}, {"type": "ContinuousInput", "key": "base_eq", "unit": null, "bounds": [1.0, 2.5]}, {"type": "ContinuousInput", "key": "temperature", "unit": null, "bounds": [30.0, 100.0]}, {"type": "ContinuousInput", "key": "t_res", "unit": null, "bounds": [60.0, 1800.0]}]}, "outputs": {"type": "Outputs", "features": [{"type": "ContinuousOutput", "key": "yield", "objective": {"type": "MaximizeObjective", "w": 

In [29]:
# Load it from the spec
surrogate_data = parse_obj_as(AnySurrogate, json.loads(jspec))
# Map it 
surrogate = surrogates.map(surrogate_data)
# Fit it
surrogate.fit(experiments=experiments)
# dump it
dump = surrogate.dumps()
# predict with it
df_predictions = surrogate.predict(experiments)
# transform to spec
predictions = surrogate.to_predictions(predictions=df_predictions)

In [30]:
surrogate_data = parse_obj_as(AnySurrogate, json.loads(jspec))
surrogate = surrogates.map(surrogate_data)
surrogate.loads(dump)

# predict with it
df_predictions2 = surrogate.predict(experiments)
# transform to spec
predictions2 = surrogate.to_predictions(predictions=df_predictions2)

# check for equality
predictions==predictions2

True